In [5]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(  
    temperature=0,
    groq_api_key='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',
    model="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was")
print(response.content)

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission.

Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on another celestial body.

He was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the mission.


In [7]:
!pip install -qU langchain_community beautifulsoup4


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2959878/system-development-engineer-ii-realm-hyd")
page_data=loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


'System Development Engineer II, REALM-HYD - Job ID: 2959878 | Amazon.jobs\nSkip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×System Development Engineer II, REALM-HYDJob ID: 2959878 | ADCI HYD 13 SEZApply nowDESCRIPTIONThe REALM Team is looking for a System Dev Engineer who will act as first level support for all incoming production software application issues dealing with near 100+ different services in Amazon Transportation Technology. The Systems Dev Engineer will be specializing in application support of WW Amazon transportation systems. Your solutions will impact our customers directly! This job requires you to constantly hit the ground running and your ability to learn quickly and work on disparate and overlapping tasks will define your success. High Impact production issues often require coordi

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract= prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

Here is the extracted job posting in JSON format:

```
{
  "role": "System Development Engineer II, REALM-HYD",
  "experience": "Experience in automating, deploying, and supporting large-scale infrastructure",
  "skills": ["Python", "Ruby", "Golang", "Java", "C++", "C#", "Rust", "Linux/Unix", "CI/CD pipelines"],
  "description": "The REALM Team is looking for a System Dev Engineer who will act as first level support for all incoming production software application issues dealing with near 100+ different services in Amazon Transportation Technology."
}
```


In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser= JsonOutputParser()
json_res= json_parser.parse(res.content)
json_res

{'role': 'System Development Engineer II, REALM-HYD',
 'experience': 'Experience in automating, deploying, and supporting large-scale infrastructure',
 'skills': ['Python',
  'Ruby',
  'Golang',
  'Java',
  'C++',
  'C#',
  'Rust',
  'Linux/Unix',
  'CI/CD pipelines'],
 'description': 'The REALM Team is looking for a System Dev Engineer who will act as first level support for all incoming production software application issues dealing with near 100+ different services in Amazon Transportation Technology.'}

In [11]:
import pandas as pd

df=pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://rishithadaivamdinne.github.io/autopitc...
1,"Angular,.NET, SQL Server",https://rishithadaivamdinne.github.io/autopitc...
2,"Vue.js, Ruby on Rails, PostgreSQL",https://rishithadaivamdinne.github.io/autopitc...
3,"Python, Django, MySQL",https://rishithadaivamdinne.github.io/autopitc...
4,"Java, Spring Boot, Oracle",https://rishithadaivamdinne.github.io/autopitc...
5,"Flutter, Firebase, GraphQL",https://rishithadaivamdinne.github.io/autopitc...
6,"WordPress, PHP, MySQL",https://rishithadaivamdinne.github.io/autopitc...
7,"Magento, PHP, MySQL",https://rishithadaivamdinne.github.io/autopitc...
8,"React Native, Node.js, MongoDB",https://rishithadaivamdinne.github.io/autopitc...
9,"iOS, Swift, Core Data",https://rishithadaivamdinne.github.io/autopitc...


In [19]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collectionone = client.get_or_create_collection(name="portfoliotwo")

if not collectionone.count():
    for _, row in df.iterrows():
        collectionone.add(documents=row["Techstack"], metadatas={"links": row["Links"]}, ids=[str(uuid.uuid4())])

In [20]:
links=collectionone.query(query_texts=["Experience in Python","Expertise in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/ml-python-tensorflow.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/python-django-mysql.html  '}],
 [{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/react-node-mongodb.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/react-native-node-mongo.html  '}]]

In [21]:
job=json_res

In [22]:
links=collectionone.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/ml-python-tensorflow.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/python-django-mysql.html  '}],
 [{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/vue-rails-postgres.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/python-django-mysql.html  '}],
 [{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/ios-swift-arkit.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/ml-python-tensorflow.html  '}],
 [{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/java-springboot-oracle.html  '},
  {'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/android-java-room.html  '}],
 [{'links': 'https://rishithadaivamdinne.github.io/autopitch-tech-portfolios/ml-python-tensorflow.html  '},
  {'links': 'https://rishithadaiva

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rishitha Daivamdinne, a pre-final year Computer Science and IoT engineering student with a CGPA of 9.22. 
    You are passionate about solving real-world problems with technology and have experience in areas such as 
    software development, AI/ML, and embedded systems. You are actively seeking opportunities to contribute 
    meaningfully to innovative teams and gain hands-on experience in the industry.

    
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Rishitha 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Rishitha's portfolio: {link_list}
        Remember you are Rishitha Daivamdinne. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for System Development Engineer II, REALM-HYD Role

Dear Hiring Manager,

I came across the System Development Engineer II, REALM-HYD role at Amazon Transportation Technology, and I'm excited to apply for this position. With my strong foundation in computer science and IoT engineering, I'm confident that I can contribute to the REALM Team's mission to support large-scale infrastructure.

As a pre-final year student with a CGPA of 9.22, I've developed a solid understanding of programming languages such as Python, Java, and C++. My experience in software development, AI/ML, and embedded systems has equipped me with the skills to automate, deploy, and support complex systems.

I'm particularly drawn to this role because of the opportunity to work with a wide range of services (near 100+) in Amazon Transportation Technology. My strong problem-solving skills and passion for technology make me an ideal fit for this position.

To showcase my capabilities, I'd like to shar